In [1]:
from dotenv import load_dotenv
import fmpsdk
import pandas as pd
import os

load_dotenv()
apikey = os.environ.get("apikey")

snp500 = pd.DataFrame(fmpsdk.sp500_constituent(apikey))

In [ ]:
sectors = snp500['sector'].unique().tolist()
sector_constant_list = [(sector_key.upper().replace(" ", "_") + '=\'' + sector_key + '\'')
                        for
                        sector_key in
                        sectors]

with open("sector_constant.txt", "a") as f:
    print(*sector_constant_list, sep="\n", file=f)

In [ ]:
import seaborn as sns
colors = sns.color_palette("cubehelix", len(sectors))
sector_color_mapper = {k: v for k,v in zip(sectors, colors)}



In [ ]:
sub_sectors = snp500['subSector'].unique().tolist()
sector_constant_list = [('PROFILE_INDUSTRY_' +
                         sector_key.upper().
                         replace("&", "").
                         replace("-", "_").replace(", ", "").replace(" ", "_").replace("__",
                                                                                       "_") + '=\'' + sector_key + '\'')
                        for
                        sector_key in
                        sub_sectors]
with open("sub_sector_constant.txt", "w") as f:
    print(*sector_constant_list, sep="\n", file=f)

In [ ]:
title = 'company-profile'
from datetime import date

today = date.today()

file_name = title + '-' + (today.strftime("%Y%m%d")) + '.csv'

df_snp500 = snp500.copy()
df_snp500.set_index('symbol', inplace=True)
df_snp500.to_csv(file_name, index=True)

In [ ]:
title = 'financial-ratios'
period = 'quarter'

from tqdm import tqdm
import pandas as pd
from datetime import date

symbols = snp500['symbol']
fmp_financial_ratios = []
for symbol in tqdm(symbols):
    fmp_financial_ratios += fmpsdk.financial_ratios(apikey=apikey, symbol=symbol, limit=-1, period=period)

df_financial_ratios = pd.DataFrame(fmp_financial_ratios, index=None)

today = date.today()
fmp_api_name = title + '-' + period
file_name = fmp_api_name + '-' + (today.strftime("%Y%m%d")) + '.csv'
df_financial_ratios.to_csv(file_name, index=False)

In [ ]:
title = 'economic-indicators'

from datetime import date

today = date.today()
from urllib.request import urlopen
import certifi
import json
import os
from dotenv import load_dotenv

from_date = '1962-01-01'
to_date = today.strftime("%Y-%m-%d")
economic_indicators = ['CPI', 'inflationRate', 'inflation', 'unemploymentRate']

load_dotenv()
apikey = os.environ.get("apikey")


def get_jsonparsed_data(url):
    response = urlopen(url, cafile=certifi.where())
    data = response.read().decode("utf-8")
    return json.loads(data)


def economic_indicator(topic: str):
    url = "https://financialmodelingprep.com/api/v4/economic?name={}&from={}&to={}&apikey={}".format(
        topic, from_date, to_date, apikey)
    fmp_economic_indicator = get_jsonparsed_data(url)
    return [{'date': item['date'], topic: item['value']} for item in fmp_economic_indicator]


result = []
for topic in economic_indicators:
    result += (economic_indicator(topic=topic))

df_economic_indicators = pd.DataFrame(result)
df_economic_indicators.set_index('date', inplace=True)
df_economic_indicators.index = pd.to_datetime(df_economic_indicators.index)
df_economic_indicators = df_economic_indicators.sort_index()

import numpy as np

df_economic_indicators = df_economic_indicators.groupby(df_economic_indicators.index).sum()
df_economic_indicators.replace(0, np.nan, inplace=True)
file_name = title + '-' + (today.strftime("%Y%m%d")) + '.csv'
df_economic_indicators.to_csv(file_name, index=True)

In [ ]:
title = 'market-index'
from datetime import date

today = date.today()
from dotenv import load_dotenv
import fmpsdk
import pandas as pd
import os

load_dotenv()
apikey = os.environ.get("apikey")

from_date = '1962-01-01'
to_date = today.strftime("%Y-%m-%d")
index_list = pd.DataFrame(fmpsdk.available_indexes(apikey=apikey))
target_index_list = ['^GSPC', '^TYX', '^FVX', '^TNX']

target_index_history = []
for target_index in target_index_list:
    for item in fmpsdk.historical_price_full(apikey=apikey, symbol=target_index, from_date=from_date, to_date=to_date):
        target_index_history.append({'symbol': target_index, 'date': item['date'], 'price': item['adjClose']})

df_market_index = pd.DataFrame(target_index_history)
df_market_index = df_market_index.pivot(index='date', columns='symbol', values='price')
file_name = title + '-' + (today.strftime("%Y%m%d")) + '.csv'
df_market_index.to_csv(file_name, index=True)

In [ ]:
title = 'market-capitalization'
from dotenv import load_dotenv
import fmpsdk
import pandas as pd
from tqdm import tqdm
import os

load_dotenv()

apikey = os.environ.get("apikey")

result = []
for symbol in tqdm(snp500['symbol']):
    result += fmpsdk.historical_market_capitalization(symbol=symbol, apikey=apikey, limit=-1)

from datetime import date

today = date.today()
df_market_capitalization = pd.DataFrame(result)
file_name = title + '-' + (today.strftime("%Y%m%d")) + '.csv'
df_market_capitalization.to_csv(file_name, index=False)

In [ ]:
import pandas as pd
from datetime import date

title = 'market-capitalization'

today = date.today()
df_market_capitalization = pd.read_csv('market-capitalization-20220828.csv')[::4]
file_name = title + '-' + (today.strftime("%Y%m%d")) + '.csv'
df_market_capitalization.to_csv(file_name, index=False)

In [ ]:
title = 'etc-history'
from dotenv import load_dotenv
import fmpsdk
import pandas as pd
from tqdm import tqdm
import os

load_dotenv()
apikey = os.environ.get("apikey")

df_bitcoin_history = pd.DataFrame(fmpsdk.historical_price_full(symbol='BTCUSD', apikey=apikey)).set_index('date').sort_index()[
    'adjClose']
df_bitcoin_history.rename('BTCUSD', inplace=True)

df_covid_history = pd.read_csv('covid-case.csv', index_col='date')['cases']
df_covid_history.rename('covidCases', inplace=True)
df_etc = pd.concat([df_covid_history, df_bitcoin_history], axis=1, ).sort_index()

from datetime import date

today = date.today()
file_name = title + '-' + (today.strftime("%Y%m%d")) + '.csv'
df_etc.to_csv(file_name, index=True)

In [ ]:
title = 'financial-statements'

from dotenv import load_dotenv
import fmpsdk
import pandas as pd
from tqdm import tqdm
import os

load_dotenv()
apikey = os.environ.get("apikey")

financial_statements = dict()
for period in ['annual', 'quarter']:
    fmp_income_statement = []
    fmp_balance_sheet = []
    fmp_cashflow_statement = []
    for symbol in tqdm(snp500['symbol']):
        fmp_income_statement += fmpsdk.income_statement(apikey=apikey, symbol=symbol, period=period, limit=-1)
        fmp_balance_sheet += fmpsdk.balance_sheet_statement(apikey=apikey, symbol=symbol, period=period, limit=-1)
        fmp_cashflow_statement += fmpsdk.cash_flow_statement(apikey=apikey, symbol=symbol, period=period, limit=-1)
    financial_statements.update({period: [fmp_income_statement, fmp_balance_sheet, fmp_cashflow_statement]})

from datetime import date

today = date.today()
for period in ['annual', 'quarter']:
    file_name = title + '-' + period + '-' + (today.strftime("%Y%m%d")) + '.csv'
    df_financial_statements = pd.concat([pd.DataFrame(item) for item in financial_statements[period]], axis=1)
    df_financial_statements.to_csv(file_name, index=False)

columns = df_financial_statements.columns
with open("financial_statements_list.txt", "a") as f:
    print(*columns, sep="\n", file=f)

In [ ]:
title = 'price-history'
from dotenv import load_dotenv
import fmpsdk
import pandas as pd
from tqdm import tqdm
from datetime import date
import os

load_dotenv()
apikey = os.environ.get("apikey")

snp500 = pd.DataFrame(fmpsdk.sp500_constituent(apikey))

today = date.today()

from_date = '1962-01-01'
to_date = today.strftime("%Y-%m-%d")
result = []
for symbol in tqdm(snp500['symbol']):
    dict_list = fmpsdk.historical_price_full(apikey=apikey, symbol=symbol, from_date=from_date, to_date=to_date)
    [dict_item.update({'symbol': symbol}) for dict_item in
     dict_list]
    result += dict_list

file_name = title + '-' + (today.strftime("%Y%m%d")) + '.csv'
df_result = pd.DataFrame(result)
df_result[['symbol', 'date', 'adjClose', 'volume']].to_csv(file_name,index=False)


In [ ]:
from urllib.request import urlopen
import certifi
import json

def get_jsonparsed_data(url):
    response = urlopen(url, cafile=certifi.where())
    data = response.read().decode("utf-8")
    return json.loads(data)

query = 'available-traded'
query = 'stock'
url = f"https://financialmodelingprep.com/api/v3/{query}/list?apikey={apikey}"
result = get_jsonparsed_data(url)


In [ ]:
result_sub = []

In [ ]:
df_result = pd.DataFrame(result)
df_result['exchangeShortName'].unique()
target_exchange = []
df_result[df_result['exchangeShortName'] == 'KOE']

In [ ]:
fmpsdk.historical_price_full(apikey=apikey,symbol='TSLA')

In [ ]:
page = range(100)
url = f"https://financialmodelingprep.com/api/v3/delisted-companies?page={page}&&apikey={apikey}"
result = get_jsonparsed_data(url)

In [ ]:
symbol = '053800.KQ'
url = f"https://financialmodelingprep.com/api/v3/historical-price-full/{symbol}?apikey={apikey}"
result = get_jsonparsed_data(url)

In [ ]:
symbol = '053800.KQ'
symbol = 'TSLA'

query = 'profile'
url = f"https://financialmodelingprep.com/api/v3/{query}/{symbol}?apikey={apikey}"
result = get_jsonparsed_data(url)


In [ ]:
symbol = 'A'
url = f"https://financialmodelingprep.com/api/v4/standard_industrial_classification?symbol={symbol}&apikey={apikey}"
result = get_jsonparsed_data(url)

In [ ]:
df_result = pd.DataFrame(result)
df_result['exchangeShortName'].unique()
target_exchange = []
df_result[df_result['exchangeShortName'] == 'KOE']

In [ ]:
fmpsdk.historical_price_full(apikey=apikey,symbol='TSLA')

In [ ]:
page = range(100)
url = f"https://financialmodelingprep.com/api/v3/delisted-companies?page={page}&&apikey={apikey}"
result = get_jsonparsed_data(url)

In [ ]:
symbol = '053800.KQ'
url = f"https://financialmodelingprep.com/api/v3/historical-price-full/{symbol}?apikey={apikey}"
result = get_jsonparsed_data(url)

In [ ]:
symbol = '053800.KQ'
symbol = 'TSLA'

query = 'profile'
url = f"https://financialmodelingprep.com/api/v3/{query}/{symbol}?apikey={apikey}"
result = get_jsonparsed_data(url)


In [ ]:
symbol = 'A'
url = f"https://financialmodelingprep.com/api/v4/standard_industrial_classification?symbol={symbol}&apikey={apikey}"
result = get_jsonparsed_data(url)

In [30]:
fmpsdk.historical_price_full(apikey=apikey,symbol='TSLA')

[{'date': '2022-09-06',
  'open': 272.675,
  'high': 275.99,
  'low': 265.74,
  'close': 274.42,
  'adjClose': 274.42,
  'volume': 55861686.0,
  'unadjustedVolume': 55861686.0,
  'change': 1.745,
  'changePercent': 0.64,
  'vwap': 272.05,
  'label': 'September 06, 22',
  'changeOverTime': 0.0064},
 {'date': '2022-09-02',
  'open': 281.065,
  'high': 282.35,
  'low': 269.08,
  'close': 270.21,
  'adjClose': 270.21,
  'volume': 50890438.0,
  'unadjustedVolume': 50890438.0,
  'change': -10.855,
  'changePercent': -3.862,
  'vwap': 273.88,
  'label': 'September 02, 22',
  'changeOverTime': -0.03862},
 {'date': '2022-09-01',
  'open': 272.575,
  'high': 277.58,
  'low': 266.15,
  'close': 277.16,
  'adjClose': 277.16,
  'volume': 54292008.0,
  'unadjustedVolume': 54292008.0,
  'change': 4.585,
  'changePercent': 1.682,
  'vwap': 273.63,
  'label': 'September 01, 22',
  'changeOverTime': 0.01682},
 {'date': '2022-08-31',
  'open': 280.62,
  'high': 281.25,
  'low': 271.81,
  'close': 275.61,

In [21]:
page = range(100)
url = f"https://financialmodelingprep.com/api/v3/delisted-companies?page={page}&&apikey={apikey}"
result = get_jsonparsed_data(url)

C:\Users\JBS\AppData\Local\Temp\ipykernel_36228\3801304595.py:6: DeprecationWarning: cafile, capath and cadefault are deprecated, use a custom context instead.
  response = urlopen(url, cafile=certifi.where())


In [31]:
symbol = '053800.KQ'
url = f"https://financialmodelingprep.com/api/v3/historical-price-full/{symbol}?apikey={apikey}"
result = get_jsonparsed_data(url)

C:\Users\JBS\AppData\Local\Temp\ipykernel_36228\3801304595.py:6: DeprecationWarning: cafile, capath and cadefault are deprecated, use a custom context instead.
  response = urlopen(url, cafile=certifi.where())


In [42]:
symbol = '053800.KQ'
symbol = 'TSLA'

query = 'profile'
url = f"https://financialmodelingprep.com/api/v3/{query}/{symbol}?apikey={apikey}"
result = get_jsonparsed_data(url)


C:\Users\JBS\AppData\Local\Temp\ipykernel_36228\3801304595.py:6: DeprecationWarning: cafile, capath and cadefault are deprecated, use a custom context instead.
  response = urlopen(url, cafile=certifi.where())


In [37]:
symbol = 'A'
url = f"https://financialmodelingprep.com/api/v4/standard_industrial_classification?symbol={symbol}&apikey={apikey}"
result = get_jsonparsed_data(url)

C:\Users\JBS\AppData\Local\Temp\ipykernel_36228\3801304595.py:6: DeprecationWarning: cafile, capath and cadefault are deprecated, use a custom context instead.
  response = urlopen(url, cafile=certifi.where())
